In [ ]:
import rasterio
from rasterio.plot import show as rastershow
import sqlite3
import pandas as pd
import geopandas as gpd
import os

from matplotlib import pyplot as plt

A quick check to see if the generated polygons match expectations (this was without any finetuning)

In [ ]:
test_tile = '/work/ka1176/shared_data/2024-ufz-deeptree/example_tiles/LVermGeo_DOP20/dop20rgbi_32_702_5706_2_st_2022.tif'
test_sqlite = '/work/ka1176/caroline/gitlab/TreeCrownDelineation/results/inference_halle/2024-09-20_13-11-47/test.sqlite'

In [ ]:
def geoframe_from_sqlite(dbfile, col='test'):
    '''
    Create a GeoDataFrame with the geometries in the SQLITE database

    Arguments:
    dbfile (str): path to SQLITE database 
    '''

    if not os.path.exists(dbfile):
        print('No such file', dbfile)
        return None
    
    con = sqlite3.connect(dbfile)
    cur = con.cursor()
    try:
        df = pd.read_sql_query(f"SELECT * from {col}", con)
    except:
        print('Query failed for file', dbfile)
        return None
        
    con.close()
    geo = gpd.GeoSeries.from_wkb(df['GEOMETRY']).to_frame(f'geometry')

    return geo

In [ ]:
geoframe = geoframe_from_sqlite(test_sqlite)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
rastershow(rasterio.open(test_tile), ax=ax)
geoframe.plot(ax=ax)